# Pyaflowa

`Pyaflowa` can be used standalone, and also provides the necessary interface to work with SeisFlows3. When used within the SeisFlows3 preprocess module, Pyaflowa reduces the overhead required to include Pyatoa functionality into a SeisFlows3 inversion.

## Standalone (w/ SPECFEM3D)

Let's say you want to run forward and adjoint simulations to generate synthetic seismograms and sensitivity kernels using SPECFEM3D. To make this happen, a user will need some specific pieces that correspond to the problem at hand.

* Forward simulations: DATA/CMTSOLUTION, DATA/STATIONS  
* Adjoint simulations: DATA/STATIONS_ADJOINT, /sem/\*.adj 

We can use `Pyatoa` to prepare the necessary files for SPECFEM3D forward simulatinos, and then use `Pyaflowa` to: 1) compare synthetic seismograms with real data for a number of stations and generate adjoint sources which can be fed directly back into a SPECFEM3D adjoint simulation. 

### Prep: Generate CMTSOLUTION

First we must generate the source file used to represent the moment tensor in our earthquake simulation. In this example, we'll use my favorite New Zealand [example event, 2018p130600](https://www.geonet.org.nz/earthquake/2018p130600), an M5.2 that occurred in the central North Island, New Zealand while I was doing my PhD in Wellington.

With `ObsPy` + `Pyatoa` we'll be able to gather this event from FDSN webservices and save the resulting moment tensor and event information to the required [CMTSOLUTION format](https://www.globalcmt.org/CMTsearch.html) expected by SPECFEM3D. 

In [1]:
import os
from pyatoa import append_focal_mechanism
from obspy.clients.fdsn import Client

In [2]:
# We will just move to an empty directory for this example problem
cwd = os.getcwd()  # Make sure we can get back to the docs directory
print(cwd)

docs_path = os.path.abspath("../tests/test_data/docs_data/pyaflowa_doc")
if not os.path.exists(docs_path):
    os.mkdir(docs_path)
os.chdir(docs_path)
print(os.getcwd())

/home/bchow/REPOSITORIES/pyatoa/pyatoa/docs
/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc


In [3]:
# Using ObsPy's FDSN Client, we can retrieve event information using an event id
c = Client("GEONET")
cat = c.get_events(eventid="2018p130600")

# Using Pyatoa, we can append a focal mechanism from the GeoNet moment tensor catalog
cat[0] = append_focal_mechanism(cat[0], client="GEONET")

# ObsPy has built in support for writing CMTSOLUTION files expected by SPECFEM3D
cat.write("CMTSOLUTION", format="CMTSOLUTION")

/home/bchow/miniconda3/envs/docs/lib/python3.7/site-packages/obspy/io/cmtsolution/core.py:370: UserWarning: No body wave magnitude found. Will be replaced by the first magnitude in the event object.
  warnings.warn("No body wave magnitude found. Will be replaced by the "
/home/bchow/miniconda3/envs/docs/lib/python3.7/site-packages/obspy/io/cmtsolution/core.py:376: UserWarning: No surface wave magnitude found. Will be replaced by the first magnitude in the event object.
  warnings.warn("No surface wave magnitude found. Will be replaced by "
/home/bchow/miniconda3/envs/docs/lib/python3.7/site-packages/obspy/io/cmtsolution/core.py:397: UserWarning: Could not find a centroid origin. Will instead assume that the preferred or first origin is the centroid origin.
  warnings.warn("Could not find a centroid origin. Will instead "
/home/bchow/miniconda3/envs/docs/lib/python3.7/site-packages/obspy/io/cmtsolution/core.py:410: UserWarning: Hypocentral origin will be identical to the centroid one.
 

In [4]:
# Lets just have a look at the file that's been created, which is a CMTSOLUTION that is ready
# to be used in SPECFEM3D
!cat "CMTSOLUTION"

 PDE 2018 02 18 07 43 48.13  -39.9490  176.2995  20.6 5.2 5.2 NORTH ISLAND, NEW ZEALAND
event name:           0DBD19
time shift:           0.0000
half duration:        0.6989
latitude:           -39.9490
longitude:          176.2995
depth:               20.5946
Mrr:           -2.479380E+23
Mtt:            1.314880E+23
Mpp:            1.164500E+23
Mrt:            5.032500E+22
Mrp:            6.607700E+22
Mtp:            9.359300E+22


### Prep: Generate STATIONS file

SPECFEM3D also requires a STATIONS file which defines the locations of receivers for simulation output. 
As in Step 1 we'll generate a list of stations using `ObsPy` and write them into the required STATIONS file using `Pyatoa` and a corresponding obspy.Inventory object.

> **__NOTE:__**  
In the ObsPy function get_stations(), the reasoning behind the following arguments provided:
* __network = "NZ"__ refers to the code for New Zealand's permament seismic netnwork
* __station = "??Z"__ means we only want 3 letter station codes that end in Z, which GeoNet usually reserves for broadband seismometers
* __channel = "HH?"__ refers to a broadband (first 'H') seismometer (second 'H'), for any available component (wildcard '?'), usually N/E/Z. This follows [SEED naming convention](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/).
* This __min__ and __max latitude / longitude__ defines a small region where we want to search for stations

In [5]:
from pyatoa import write_stations

In [6]:
inv = c.get_stations(network="NZ", station="??Z", channel="HH?",
                     minlatitude=-41, maxlatitude=-39,
                     minlongitude=173, maxlongitude=176)
write_stations(inv, fid="STATIONS")

/home/bchow/miniconda3/envs/docs/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))


In [7]:
# Let's have a look at the stations we picked up from FDSN
print(inv)

Inventory created at 2022-03-02T06:13:37.000000Z
	Created by: Delta
		    
	Sending institution: GeoNet (WEL(GNS_Test))
	Contains:
		Networks (1):
			NZ
		Stations (4):
			NZ.MRZ (Mangatainoka River)
			NZ.TSZ (Takapari Road)
			NZ.VRZ (Vera Road)
			NZ.WAZ (Wanganui)
		Channels (0):



In [8]:
# And lets have a look at the STATIONS file that's been created
!cat "STATIONS"

   MRZ    NZ    -40.6605    175.5785    0.0    0.0
   TSZ    NZ    -40.0586    175.9611    0.0    0.0
   VRZ    NZ    -39.1243    174.7585    0.0    0.0
   WAZ    NZ    -39.7546    174.9855    0.0    0.0


### Forward Simulation: Generate synthetics using SPECFEM3D [external]

Unfortunately this cannot be shown in a Jupyter notebook as generating synthetics requires interfacing with the SPECFEM3D code, which usually takes place on a cluster. In this example we assume this step has been completed successfully, with resultant synthetic waveforms produced by SPECFEM3D for the given event and stations defined above. 

> **__NOTE:__**  
Output synthetic seismograms are expected to be formatted as two-column ASCII files, which I have pre-generated for this example. File names follow the expected output from SPECFEM3D. Adherance to this format is very important for running Pyaflowa. 

> **__NOTE:__**
By default synthetic waveform data is expected to be separated by event ID, e.g., PATH/TO/SYNTHETICS/{EVENT_ID}/*semd

In [9]:
# Let's copy the premade synthetic data into our current working directory
!ls ../../synthetics
!mkdir -p synthetics
!cp -r ../../synthetics/201?p?????? ./synthetics

2012p242656  2018p130600  NZ.BFZ.BXE.semd  NZ.BFZ.BXN.semd  NZ.BFZ.BXZ.semd


In [10]:
!head ./synthetics/2018p130600/NZ.MRZ.BX?.semd

==> ./synthetics/2018p130600/NZ.MRZ.BXE.semd <==
  -20.0000000       0.00000000    
  -19.9850006       0.00000000    
  -19.9699993       0.00000000    
  -19.9549999       0.00000000    
  -19.9400005       0.00000000    
  -19.9249992       0.00000000    
  -19.9099998       0.00000000    
  -19.8950005       0.00000000    
  -19.8799992       0.00000000    
  -19.8649998       0.00000000    

==> ./synthetics/2018p130600/NZ.MRZ.BXN.semd <==
  -20.0000000       0.00000000    
  -19.9850006       0.00000000    
  -19.9699993       0.00000000    
  -19.9549999       0.00000000    
  -19.9400005       0.00000000    
  -19.9249992       0.00000000    
  -19.9099998       0.00000000    
  -19.8950005       0.00000000    
  -19.8799992       0.00000000    
  -19.8649998       0.00000000    

==> ./synthetics/2018p130600/NZ.MRZ.BXZ.semd <==
  -20.0000000       0.00000000    
  -19.9850006       0.00000000    
  -19.9699993       0.00000000    
  -19.9549999       0.00000000    
  -19.94000

### Pyaflowa's directory structure

`Pyaflowa` abstracts away the enigmatic inner machinations of `Pyatoa`. To do so it manages an internal directory structure to search for inputs and store outputs.

When used standalone, `Pyaflowa` creates its own directory structure within a given working directory. When used in conjunction with SeisFlows3, `Pyaflowa` will work within the preset internal directory structure of SeisFlows3 (see Pyaflowa + SeisFlows3).

Let's start by initiating `Pyaflowa`. As with any usage of Pyatoa, a Config object is required to define internally used parameters which will inturn be used to control gathering, waveform processing, and misfit quantification.

In [11]:
from pyatoa import Pyaflowa, Config

In [12]:
cfg = Config(iteration=1, step_count=0, client="GEONET", min_period=10, max_period=30,
             pyflex_preset="nznorth_10-30s")

pf = Pyaflowa(structure="standalone", workdir="./", config=cfg)

In [13]:
# We can take a look at Pyaflowa's DEFAULT internal directory structure with the path_structure attribute
pf.path_structure

cwd          : './'
data         : './input/DATA'
datasets     : './datasets'
figures      : './figures'
logs         : './logs'
ds_file      : './datasets/{source_name}.h5'
stations_file: './{source_name}/STATIONS'
responses    : './input/responses'
waveforms    : './input/waveforms'
synthetics   : './input/synthetics/{source_name}'
adjsrcs      : './adjsrcs/{source_name}'
event_figures: './figures/{source_name}'

If you want to change the Pyaflowa directory structure from the default values shown above, you can simply pass the keys of the `path_structure` attribute as keyword arguments in the initialization of Pyaflowa. Let's generate a non-standard path structure to point to our existing data.

In [14]:
# Make sure we're in the correct directory so that we don't start making dir. randomly
assert os.path.basename(os.getcwd()) == "pyaflowa_doc"

# Custom set directory structure
base_path = os.getcwd()
kwargs = {"workdir": base_path,
          "synthetics": os.path.join(base_path, "synthetics", "{source_name}"),
          "stations_file": os.path.join(base_path, "STATIONS"),
          "data": base_path,
          "datasets": base_path,
         }

pf = Pyaflowa(structure="standalone", config=cfg, **kwargs)
pf.path_structure

cwd          : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
data         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
datasets     : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
figures      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/figures'
logs         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/logs'
ds_file      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/{source_name}.h5'
stations_file: '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/STATIONS'
responses    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/responses'
waveforms    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/waveforms'
synthetics   : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/doc

----------------------

### The IO (input/output) class

By running the Pyaflowa.setup() function, Pyaflowa will make the required directory structure defined above. It will also return an `IO` object. This internally used object store information related to paths, configurations and processing.

The user does __not__ need to interact with the `IO` object, but we can take a look at it for clarity. It contains the internal directory structure used by `Pyaflowa`, the `Config` object which will control all of the Manager processing that will take place, and internal attributes which keep track of how processing occurs.

In [15]:
io = pf.setup(source_name="2018p130600")

In [16]:
for key, val in io.items():
    print(key, val)

paths cwd          : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
data         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
datasets     : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
figures      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/figures'
logs         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/logs'
ds_file      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/2018p130600.h5'
stations_file: '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/STATIONS'
responses    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/responses'
waveforms    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/waveforms'
synthetics   : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data

-----------------------
### Running Pyaflowa (gather and process waveforms)

Great, we're all set up to run `Pyaflowa`. Internally `Pyaflowa` knows the event, path structure and stations that we want to use for misfit quantification. Now when we run it, `Pyaflowa` will instantiate `Manager` classes, attempt to gather data from disk or from web services, preprocess data and synthetic waveforms according to the `Config` object, and generate misfit windows and adjoint sources.

The example code block below is a an example of what Pyaflowa is doing under the hood: it simply abstracts commands that are used to run processing for multiple stations. It also contains a few internal checks to make sure unexpected errors don't throw the processing step off the rails.::

    from pyasdf import ASDFDataSet 
    from pyatoa import Manager

    with ASDFDataSet(io.paths.ds_file) as ds:
        mgmt = Manager(ds=ds, config=io.config)
        for code in ["NZ.BFZ.*.HH?"]:
            mgmt.gather(code=code)
            mgmt.flow()

In [17]:
pf.process_event(source_name="2018p130600", loc="*", cha="HH?")

[2022-03-03 11:02:02] - pyatoa - DEBUG: gathering event
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for event info
[2022-03-03 11:02:02] - pyatoa - DEBUG: matching event found: A8E579
[2022-03-03 11:02:02] - pyatoa - INFO: 

NZ.MRZ.*.HH?

[2022-03-03 11:02:02] - pyatoa - DEBUG: gathering event
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for event info
[2022-03-03 11:02:02] - pyatoa - DEBUG: matching event found: A8E579
[2022-03-03 11:02:02] - pyatoa - INFO: gathering data for NZ.MRZ.*.HH?
[2022-03-03 11:02:02] - pyatoa - INFO: gathering observed waveforms
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for observations
[2022-03-03 11:02:02] - pyatoa - INFO: matching observed waveforms found
[2022-03-03 11:02:02] - pyatoa - INFO: gathering StationXML
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for station info
[2022-03-03 11:02:02] - pyatoa - INFO: matching StationXML found
[2022-03-03 11:02:02] - pyatoa - INFO: saved to AS

0.138628125

### Inspect Pyaflowa outputs

Iwe have a look at the work directory, we can see the outputs of the Pyaflowa workflow, which will be:
* An ASDFDataSet with waveforms, metadata, misfit windows and adjoint sources
* Waveform figures for all the stations processed
* Adjoint source ASCII files (.adj) required for a SPECFEM3D adjoint simulation
* STATIONS_ADJOINT file required for a SPECFEM3D adjoint simulation
* The output log which shows the 

In [18]:
# Here is the working directory with all the inputs and outputs
!ls

2012p242656.h5	adjsrcs      figures  logs	STATIONS_ADJOINT
2018p130600.h5	CMTSOLUTION  input    STATIONS	synthetics


In [19]:
# Each event will output adjoint source files that can be fed directly into SPECFEM3D
!ls adjsrcs/2018p130600

NZ.MRZ.BXE.adj	NZ.MRZ.BXZ.adj	NZ.VRZ.BXN.adj	NZ.WAZ.BXE.adj	NZ.WAZ.BXZ.adj
NZ.MRZ.BXN.adj	NZ.VRZ.BXE.adj	NZ.VRZ.BXZ.adj	NZ.WAZ.BXN.adj


In [20]:
# Adjoint source files are created as two-column ASCII files, in the same manner as the synthetics 
# generated by SPECFEM3D
!head adjsrcs/2018p130600/NZ.MRZ.*.adj

==> adjsrcs/2018p130600/NZ.MRZ.BXE.adj <==
-2.000000000000000000e+01 0.000000000000000000e+00
-1.998499999999999943e+01 0.000000000000000000e+00
-1.996999999999999886e+01 0.000000000000000000e+00
-1.995499999999999829e+01 0.000000000000000000e+00
-1.994000000000000128e+01 0.000000000000000000e+00
-1.992500000000000071e+01 0.000000000000000000e+00
-1.991000000000000014e+01 0.000000000000000000e+00
-1.989499999999999957e+01 0.000000000000000000e+00
-1.987999999999999901e+01 0.000000000000000000e+00
-1.986499999999999844e+01 0.000000000000000000e+00

==> adjsrcs/2018p130600/NZ.MRZ.BXN.adj <==
-2.000000000000000000e+01 0.000000000000000000e+00
-1.998499999999999943e+01 0.000000000000000000e+00
-1.996999999999999886e+01 0.000000000000000000e+00
-1.995499999999999829e+01 0.000000000000000000e+00
-1.994000000000000128e+01 0.000000000000000000e+00
-1.992500000000000071e+01 0.000000000000000000e+00
-1.991000000000000014e+01 0.000000000000000000e+00
-1.989499999999999957e+01 0.000000000000000000

In [21]:
# A composite PDF of all waveform figures for each source-receiver pair will be generated
# for the user to quickly evaluate data-synthetic misfit graphically
!ls figures/2018p130600

i01s00_2018p130600.pdf


In [22]:
# This doesn't work, image needs to be stored within the dir. that I opened jupyter notebook with
# from IPython.display import IFrame
# IFrame("figures/2018p130600/i01s00_2018p130600.pdf", width=600, height=300)

In [23]:
# Text log files help the user keep track of all processing steps, and misfit information
!ls logs

i01s00_2012p242656.log	i01s00_2018p130600.log


In [24]:
!cat logs/i01s00_2018p130600.log

[2022-03-03 11:02:02] - pyatoa - INFO: 

NZ.MRZ.*.HH?

[2022-03-03 11:02:02] - pyatoa - DEBUG: gathering event
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for event info
[2022-03-03 11:02:02] - pyatoa - DEBUG: matching event found: A8E579
[2022-03-03 11:02:02] - pyatoa - INFO: gathering data for NZ.MRZ.*.HH?
[2022-03-03 11:02:02] - pyatoa - INFO: gathering observed waveforms
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for observations
[2022-03-03 11:02:02] - pyatoa - INFO: matching observed waveforms found
[2022-03-03 11:02:02] - pyatoa - INFO: gathering StationXML
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for station info
[2022-03-03 11:02:02] - pyatoa - INFO: matching StationXML found
[2022-03-03 11:02:02] - pyatoa - INFO: saved to ASDFDataSet
[2022-03-03 11:02:02] - pyatoa - INFO: gathering synthetic waveforms
[2022-03-03 11:02:02] - pyatoa - INFO: searching ASDFDataSet for synthetics
[2022-03-03 11:02:02] - pyatoa - INFO: searchi

### Multi-processing with Pyaflowa

Pyaflowa allows for multi-processing using Python's concurrent.futures. This means that multiple events can be processed in parallel, potentially allowing for large speed up when running waveform processing. We'll include another event [2012p242656](https://www.geonet.org.nz/earthquake/2012p242656), in our work directory and show how simple it is to use this functionality.

In [25]:
!ls synthetics/2012p242656

NZ.MRZ.BXE.semd  NZ.TSZ.BXE.semd  NZ.VRZ.BXE.semd  NZ.WAZ.BXE.semd
NZ.MRZ.BXN.semd  NZ.TSZ.BXN.semd  NZ.VRZ.BXN.semd  NZ.WAZ.BXN.semd
NZ.MRZ.BXZ.semd  NZ.TSZ.BXZ.semd  NZ.VRZ.BXZ.semd  NZ.WAZ.BXZ.semd


In [26]:
# Here we initiate almost the same key word arguments, however we add a formatting statement in the synthetics
# kwarg so that Pyaflowa knows which directory to search when looking for synthetics.
# Here the STATION FILE is the same
pf.path_structure

cwd          : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
data         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
datasets     : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc'
figures      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/figures'
logs         : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/logs'
ds_file      : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/{source_name}.h5'
stations_file: '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/STATIONS'
responses    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/responses'
waveforms    : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/docs_data/pyaflowa_doc/input/waveforms'
synthetics   : '/home/bchow/REPOSITORIES/pyatoa/pyatoa/tests/test_data/doc

In [27]:
# Now, all we need to do to invoke multiprocessing is input a list of source names to the run function
# !!! This doesn't work :(
# pf.multi_event_process(source_names=["2018p130600", "2012p242656"], cha="HH?", loc="*")

Unfortunately I don't think it's possible to run parallel tasks in a Jupyter notebook and I'm not prepared to put in the effort to figure out how to do this, so I hope you can take my word for it that this simply executes two parallel processing steps using the concurrent.futures multiprocessing machinery. neat!
___

## Automating earthquake-based FWT with Pyaflowa and SeisFlows3

Earthquake-based full waveform tomography (FWT) is a complicated procedure involving performing data-synthetic comparisons for large numbers of source-receiver pairs. This is ideally done in parallel as this processing step is (mostly) identical and independent for each source-receiver pair. 

There are many available workflow tools for automating FWT, but here we show the combination of `Pyatoa` and [SeisFlows3](https://github.com/bch0w/seisflows3), a Python-based workflow tool for automating full waveform tomography, adjoint tomography, or full waveform inversion (FWI) on a variety of compute systems.

`Pyaflowa` has already been implemented as a [SeisFlows3 preprocess module](https://github.com/bch0w/seisflows3/blob/master/seisflows3/preprocess/pyatoa.py). You can see Pyaflowa being called in the function [seisflows3.preprocess.prepare_eval_grad()](https://github.com/bch0w/seisflows3/blob/master/seisflows3/preprocess/pyatoa.py#L162). You can see that all of the complexity of Pyatoa is contained within itself, such that SeisFlows3 only needs to make essentially two calls to unleash all of the power of Pyatoa.

In order to interact with `Pyaflowa` during a SeisFlows3 inversion, the user edits the [SeisFlows3 parameters.yaml file](https://github.com/bch0w/pyatoa/blob/master/pyatoa/tests/test_data/test_seisflows_parameters.yaml). Since we cannot run SeisFlows3 within this jupyter notebook (yet), we will simply invoke some of the SeisFlows specific calls in Pyaflowa to investigate what is going on inside the black box. 

Let's start by implement Pyaflowa in the same fashion that the [SeisFlows3 preprocess module does](https://github.com/bch0w/seisflows3/blob/master/seisflows3/preprocess/pyatoa.py#L180). The following cell defines two functions that are slightly modified from SeisFlows3 to allow us to interact with some SeisFlows3 objects without having to invoke the entire package

In [28]:
import re, yaml

class Dict(object):
    """
    At runtime, SeisFlows defines its parameters as Dict objects, 
    Re-defined dictionary-like object for holding parameters or paths
    Allows for easier access of dictionary items, does not allow resets of
    attributes once defined, only allows updates through new dictionaries.
    """
    def __iter__(self):
        return iter(sorted(self.__dict__.keys()))

    def __getattr__(self, key):
        return self.__dict__[key]

    def __getitem__(self, key):
        return self.__dict__[key]

    def __setattr__(self, key, val):
        if key in self.__dict__:
            raise TypeError("Once defined, parameters cannot be changed.")
        self.__dict__[key] = val

    def __delattr__(self, key):
        if key in self.__dict__:
            raise TypeError("Once defined, parameters cannot be deleted.")
        raise KeyError

    def update(self, newdict):
        super(Dict, self).__setattr__('__dict__', newdict)

    def __init__(self, newdict):
        self.update(newdict)
    
    def __str__(self):
        """
        Pretty print dictionaries and first level nested dictionaries
        """
        str_ = "{"
        for key, item in vars(self).items():
            if isinstance(item, str):
                str_ += f"{key}: '{item}'\n"
            elif isinstance(item, dict):
                str_ += key + ": {\n"
                for key2, item2 in vars(self)[key].items():
                    if isinstance(item2, str):
                        str_ += f"\t{key2}: '{item2}'\n"
                    else:
                        str_ += f"\t{key2}: {item2}\n"
                str_ += "}\n"
            else:
                str_ += f"{key}: {item}\n"
        str_ += "}"
        return str_
    
    
def loadyaml(filename):
    """
    Define how the PyYaml yaml loading function behaves. 
    Replaces None and inf strings with NoneType and numpy.inf respectively
    
    :type filename: str
    :param filename: .yaml file to load in
    """
    # work around PyYAML bugs
    yaml.SafeLoader.add_implicit_resolver(
        u'tag:yaml.org,2002:float',
        re.compile(u'''^(?:
         [-+]?(?:[0-9][0-9_]*)\\.[0-9_]*(?:[eE][-+]?[0-9]+)?
        |[-+]?(?:[0-9][0-9_]*)(?:[eE][-+]?[0-9]+)
        |\\.[0-9_]+(?:[eE][-+][0-9]+)?
        |[-+]?[0-9][0-9_]*(?::[0-5]?[0-9])+\\.[0-9_]*
        |[-+]?\\.(?:inf|Inf|INF)
        |\\.(?:nan|NaN|NAN))$''', re.X),
        list(u'-+0123456789.'))

    with open(filename, 'r') as f:
        mydict = yaml.safe_load(f)

    if mydict is None:
        mydict = dict()

    # Replace 'None' and 'inf' values to match expectations
    for key, val in mydict.items():
        if val == "None":
            mydict[key] = None
        if val == "inf":
            mydict[key] = np.inf

    return mydict

In [29]:
# Here we instantiate the SeisFlows3 PAR and PATH variables, which are
# dictionaries that contain all of the necessary user-defined information
# to run an inversion
os.chdir(cwd)
PAR = loadyaml("../tests/test_data/test_seisflows_parameters.yaml")
PATH = PAR["PATHS"]
PAR.pop("PATHS")

{'SCRATCH': './scratch',
 'OUTPUT': './output',
 'SPECFEM_DATA': './sfDATA',
 'MODEL_INIT': './model_init',
 'SPECFEM_BIN': './bin',
 'DATA': None}

In [30]:
# Print out the PAR and PATH variables to see what they define
for p in [PAR, PATH]:
    for key, val in p.items():
        print(f"{key:<15}: {val}")
    print("\n")

WORKFLOW       : thrifty_maui
SOLVER         : specfem3d_maui
SYSTEM         : maui_lg
OPTIMIZE       : LBFGS
LINESEARCH     : backtrack
PREPROCESS     : pyatoa_nz
POSTPROCESS    : base
TITLE          : decluster
BEGIN          : 1
END            : 1
VERBOSE        : True
RESUME_FROM    : None
STOP_AFTER     : initialize
NTASK          : 60
NPROC          : 40
NODESIZE       : 40
WALLTIME       : 1400
TASKTIME       : 25
ENVIRONS       : 
ACCOUNT        : nesi00263
ANCIL_TASKTIME : 10
SLURMARGS      : --hint=nomultithread
CASE           : Synthetic
MATERIALS      : Elastic
DENSITY        : Constant
PRECOND        : None
NT             : 20000
DT             : 0.015
F0             : 0.1
SOURCE_PREFIX  : CMTSOLUTION
SOLVERIO       : fortran_binary
FORMAT         : ascii
COMPONENTS     : ZNE
NORMALIZE      : None
FILTER         : None
MUTE           : None
MUTECONST      : 0.0
MUTESLOPE      : 0.0
LOGGING        : debug
UNIT_OUTPUT    : DISP
MIN_PERIOD     : 8.0
MAX_PERIOD     : 20.0
ROTA

In [31]:
# pyaflowa = Pyaflowa(structure="seisflows", sfpaths=Dict(PATH), 
#                     sfpar=Dict(PAR), iteration=1, step_count=0)